# Photometry Analysis

The goal of this script is to:

1. import photometry data and crop
1. deinterleave data
1. normalize photometry data for photobleaching
1. chop up photometry data around keystrokes


## import and definition


In [ ]:
import os

import pandas as pd
from IPython.display import display
from ipywidgets import Layout, widgets

from routine.plotting import plot_events, plot_signals
from routine.processing import photobleach_correction
from routine.utilities import load_data, pool_events

In [ ]:
exp_layout = Layout(width="auto")
w_data = widgets.FileUpload(
    accept=".csv", multiple=False, description="Upload Data File", layout=exp_layout
)
w_ts = widgets.FileUpload(
    accept=".csv", multiple=False, description="Upload TimeStamp", layout=exp_layout
)
display(w_data)
display(w_ts)


In [ ]:
styling = {"style": {"description_width": "initial"}, "layout": Layout(width="40%")}
os.makedirs("./upload", exist_ok=True)
with open("./upload/data.csv", "wb") as dat:
    dat.write(w_data.value[0]["content"].tobytes())
with open("./upload/timestamp.csv", "wb") as dat:
    dat.write(w_ts.value[0]["content"].tobytes())
data = pd.read_csv("./upload/data.csv")
w_roi = widgets.SelectMultiple(
    options=data.columns, description="ROIs (Multiple Selection)", **styling
)
w_nfm = widgets.IntSlider(
    min=0,
    max=data["FrameCounter"].max(),
    step=1,
    description="Number of frames to discard",
    **styling
)
w_evt_range = widgets.IntRangeSlider(
    min=-1000, max=1000, step=1, description="Number of frames around event", **styling
)
w_led_dict = {
    i: widgets.Text(
        placeholder="e.g. 'initial', '415nm' ...",
        description="Label for LED {}".format(i),
        **styling
    )
    for i in data["LedState"].unique()
}
w_base = widgets.Text(
    placeholder="must be one of the label entered above",
    description="Name of baseline signal",
    **styling
)
display(w_roi)
display(w_nfm)
display(w_evt_range)
for w in w_led_dict.values():
    display(w)
display(w_base)


In [ ]:
IN_DATA = "./upload/data.csv"
IN_TS = "./upload/timestamp.csv"
PARAM_ROIS = list(w_roi.value)  # ["Region0G"]
PARAM_NFM_DISCARD = w_nfm.value  # 100
PARAM_EVT_RANGE = w_evt_range.value  # (-200, 400)
PARAM_LED_DICT = {
    i: w.value for i, w in w_led_dict.items()
}  # {7: "initial", 1: "415nm", 2: "470nm", 4: "560nm"}
PARAM_BASE_SIG = w_base.value  # "415nm"
assert (
    PARAM_BASE_SIG in PARAM_LED_DICT.values()
), "cannot use {} as baseline signal as it's not in LED label mapping: {}".format(
    PARAM_BASE_SIG, PARAM_LED_DICT
)


## load data


ensure equal number of frames for each channel


In [ ]:
data, ts = load_data(IN_DATA, IN_TS, PARAM_NFM_DISCARD, PARAM_LED_DICT)


## visualize raw signal


In [ ]:
plot_signals(data, PARAM_ROIS)


## correcting for photobleaching


order of operations:

1. deinterleave data by flag (LED) and save into a matrix
1. fit isosbestic signal with a biexponential decay -- the shape of this decay is a good approximation of the CONCENTRATION of GCaMP molecules underneath your fiber.
   it decreases as the photobleach.
   the amplitude, however, is tiny.
   to adjust for this, we:
1. linearly scale the fitted decay to the 470 data using robust fit.
1. divide the raw 470 data by this scale fit to get a corrected signal

note: this isn't dF/F but it is INTERNALLY reliable -- that is, you can compare the beginning of the recording to the end of the recording.
dF/F requires a good approximation of baseline.
you can use the `FP.no_led` as an underestimation of this -- or determine it empirically.
it often isn't critical to a sound analysis.


In [ ]:
data_norm = photobleach_correction(data, PARAM_ROIS, PARAM_BASE_SIG)


## visualize correction result


In [ ]:
plot_signals(data_norm, PARAM_ROIS)


## pool signals around `'Key'` events


In [ ]:
evt_df = pool_events(ts, data, PARAM_EVT_RANGE, PARAM_ROIS, event_name="Key")


## visualize signals around events


In [ ]:
plot_events(evt_df)
